<h2 style="color: #1EB382;font-weight: bold;">Redshift Integration</h2>

This notebooks guides through the ingestion of Redshift data in the Hopsworks feature store. To follow this notebook users should have an existing Redshift cluster, if not, they can follow the AWS [documentation](https://docs.aws.amazon.com/ses/latest/DeveloperGuide/event-publishing-redshift-cluster.html).

The data for this tutorial is available in CSV format [here]()
Users should create the following table in Redshift
```sql
CREATE TABLE telco(
    customer_id varchar(200),
    gender varchar(200),
    senior_citizen integer,
    partner varchar(200),
    dependents varchar(200),
    tenure integer,
    phone_service varchar(200),
    multiple_lines varchar(200),
    internet_service varchar(200),
    online_security varchar(200),
    online_backup varchar(200),
    device_protection varchar(200),
    tech_support varchar(200),
    streaming_tv varchar(200),
    streaming_movies varchar(200),
    contract varchar(200),
    paperless_billing varchar(200),
    payment_method varchar(200),
    monthly_charges double precision,
    total_charges varchar(200),
    churn varchar(200)
)
```

and populate the table using the copy command:
```sql
COPY telco
FROM 's3://bucket/telco_customer_churn.csv'
IAM_ROLE 'arn:aws:iam::xxxxxxxxx:role/role_name'
FORMAT as CSV
FILLRECORD
```

Once the data has been imported into Redshift, we can start ingesting it into the Hopsworks Feature Store. 

<h3 style="color: #1EB382;font-weight: bold;">Storage Connector</h3>

The first step to be able to ingest Redshift data in the feature store is to configure a storage connector. 
Hopsworks storage connectors are meant as a centralized place to store and manage configurations to read and write to external system. 

The Redshift connector requires you to specify the following properties. Most of them are available in the properties area of your cluster in the Redshift UI. 

- Cluster identifier: The name of the cluster

- Database driver: You can use the default JDBC Redshift Driver `com.amazon.redshift.jdbc42.Driver` (More on this later)

- Database endpoint: The endpoint for the database. Should be in the format of `[UUID].eu-west-1.redshift.amazonaws.com`

- Database name: The name of the database to query

- Database port: The port of the cluster. Defaults to 5349


There are two options to authenticate with the Redshift cluster. The first option is to configure username and password. The password is stored in the secret store and made available to all the member of the project.

Alternatively a IAM role can be configured. When configured the HSFS library will use the IAM role to acquire a temporary credential to authenticate the specified user.

With regards to the database driver, the library to interact with Redshift *is not* part of the standard Hopsworks distribution and needs to be provided by the user. The library can be downloaded here: https://docs.aws.amazon.com/redshift/latest/mgmt/configure-jdbc-connection.html#download-jdbc-driver

In [1]:
import com.logicalclocks.hsfs._
import scala.collection.JavaConversions._
import collection.JavaConverters._

import org.apache.spark.sql.{ DataFrame, Row }
import org.apache.spark.sql.types._

val connection = HopsworksConnection.builder().build();
val fs = connection.getFeatureStore();

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
36,application_1608750159023_0006,spark,idle,Link,Link


SparkSession available as 'spark'.
import com.logicalclocks.hsfs._
import scala.collection.JavaConversions._
import collection.JavaConverters._
import org.apache.spark.sql.{DataFrame, Row}
import org.apache.spark.sql.types._
connection: com.logicalclocks.hsfs.HopsworksConnection = com.logicalclocks.hsfs.HopsworksConnection@5b16ea87
fs: com.logicalclocks.hsfs.FeatureStore = FeatureStore{id=67, name='demo_fs_meb10000_featurestore', projectId=119, featureGroupApi=com.logicalclocks.hsfs.metadata.FeatureGroupApi@3b06db70}


<h3 style="color: #1EB382;font-weight: bold;">On-Demand Feature Group</h3>

To be able to read data from the Redshift table, an on-demand feature group needs to be defined.
In Hopsworks on-demand feature groups allow users to register features stored on external systems (Redshift, RDS, S3) with the Hopsworks feature store. 

Registering external features with the Hopsworks feature store has several advantages. The first one is *provenance*, from the Hopsworks Web UI users are able to track which features are stored on which external systems and how they are computed. 
Additionally HSFS (the Python/Scala library used to interact with the feature store) provides the same APIs for on-demand feature groups as for feature groups available within Hopsworks itself (Cached feature groups). Thus abstracting the complexity of configuring data ingestion.

An on-demand feature group can be defined as follow:

In [2]:
// Retrieve the storage connector defined before
val redshiftConn = fs.getStorageConnector("telco_redshift_cluster")

redshiftConn: com.logicalclocks.hsfs.StorageConnector = StorageConnector(id=1025, name=telco_redshift_cluster, accessKey=null, secretKey=null, serverEncryptionAlgorithm=null, serverEncryptionKey=null, bucket=null, clusterIdentifier=telco-redshift-cluster, databaseDriver=com.amazon.redshift.jdbc42.Driver, databaseEndpoint=cxwh6weoo4ae.eu-west-1.redshift.amazonaws.com, databaseName=dev, databasePort=5439, tableName=null, databaseUserName=awsuser, autoCreate=null, databaseGroup=null, expiration=null, databasePassword=Fabio123, sessionToken=null, connectionString=null, arguments=, storageConnectorType=REDSHIFT)


In [3]:
val telcoOnDmd = (fs.createOnDemandFeatureGroup()
                    .name("telco_redshift_scala")
                    .version(2)
                    .query("select * from telco")
                    .description("On-demand feature group for telecom customer data")
                    .storageConnector(redshiftConn)
                    .statisticsEnabled(true)
                    .build())

telcoOnDmd: com.logicalclocks.hsfs.OnDemandFeatureGroup = com.logicalclocks.hsfs.OnDemandFeatureGroup@4d876d28


In [4]:
telcoOnDmd.save()

<h3 style="color: #1EB382;font-weight: bold;">Feature Engineering</h3>

On-demand feature groups can be used directly as a source for creating training datasets. This is often the case if a company is migrating to Hopsworks and there are already feature engineering pipelines in production writing data to Redshift.

This flexibility provided by Hopsworks allows users to hit the ground running from day 1, without having to rewrite their pipelines to take advantage of the benefits the Hopsworks feature store provides.

In [5]:
telcoOnDmd.select(Seq("customer_id", "internet_service", "phone_service", "total_charges", "churn")).show(5)

+-----------+----------------+-------------+-------------+-----+
|customer_id|internet_service|phone_service|total_charges|churn|
+-----------+----------------+-------------+-------------+-----+
| 7590-VHVEG|             DSL|           No|        29.85|   No|
| 5575-GNVDE|             DSL|          Yes|       1889.5|   No|
| 3668-QPYBK|             DSL|          Yes|       108.15|  Yes|
| 7795-CFOCW|             DSL|           No|      1840.75|   No|
| 9237-HQITU|     Fiber optic|          Yes|       151.65|  Yes|
+-----------+----------------+-------------+-------------+-----+
only showing top 5 rows



On-demand feature groups can also be joined with cached feature groups to create training datasets. This document (https://docs.hopsworks.ai/generated/query_vs_dataframe/) explains in details how the HSFS joining APIs work and how they can be used to create training datasets.

If, however, Redshift contains raw data that needs to be feature engineered, users can invoke the HSFS API and retrieve a Spark DataFrame backed by the Redshift table. 

In [6]:
var sparkDf = telcoOnDmd.read()

sparkDf: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [customer_id: string, gender: string ... 19 more fields]


In [7]:
import org.apache.spark.ml.feature.StringIndexer
import org.apache.spark.ml.{Pipeline, PipelineModel}

import org.apache.spark.ml.feature.StringIndexer
import org.apache.spark.ml.{Pipeline, PipelineModel}


In [8]:
val categoricalColumns = Seq("gender", "senior_citizen","partner","dependents","phone_service","multiple_lines",
                      "internet_service", "online_security", "online_backup", "device_protection", "tech_support",
                      "streaming_tv", "streaming_movies", "contract", "paperless_billing", "payment_method", "churn")

sparkDf = sparkDf.withColumn("total_charges", $"total_charges".cast(DoubleType)).na.fill(0)

var stages = List[StringIndexer]() // stages in our Pipeline
var outputCols = List(("customer_id", "customer_id"))

for (categoricalCol <- categoricalColumns) {
    // Category Indexing with StringIndexer
    val outputCol = categoricalCol + "_index"
    val stringIndexer = new StringIndexer()
    stringIndexer.setInputCol(categoricalCol)
    stringIndexer.setOutputCol(outputCol)
    
    stages = stringIndexer :: stages
    outputCols = (categoricalCol, outputCol) :: outputCols
}

val pipeline = new Pipeline().setStages(stages.toArray)
val dataset = pipeline.fit(sparkDf).transform(sparkDf)
val telcoFgDf = dataset.selectExpr(outputCols.map(oc => oc._1  + " as " + oc._2):_*)

categoricalColumns: Seq[String] = List(gender, senior_citizen, partner, dependents, phone_service, multiple_lines, internet_service, online_security, online_backup, device_protection, tech_support, streaming_tv, streaming_movies, contract, paperless_billing, payment_method, churn)
sparkDf: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [customer_id: string, gender: string ... 19 more fields]
stages: List[org.apache.spark.ml.feature.StringIndexer] = List()
outputCols: List[(String, String)] = List((customer_id,customer_id))
pipeline: org.apache.spark.ml.Pipeline = pipeline_3f1d5b001461
dataset: org.apache.spark.sql.DataFrame = [customer_id: string, gender: string ... 36 more fields]
telcoFgDf: org.apache.spark.sql.DataFrame = [churn_index: string, payment_method_index: string ... 16 more fields]


In [9]:
telcoFgDf.show(5)

+-----------+--------------------+-----------------------+--------------+----------------------+------------------+------------------+-----------------------+-------------------+---------------------+----------------------+--------------------+-------------------+----------------+-------------+--------------------+------------+-----------+
|churn_index|payment_method_index|paperless_billing_index|contract_index|streaming_movies_index|streaming_tv_index|tech_support_index|device_protection_index|online_backup_index|online_security_index|internet_service_index|multiple_lines_index|phone_service_index|dependents_index|partner_index|senior_citizen_index|gender_index|customer_id|
+-----------+--------------------+-----------------------+--------------+----------------------+------------------+------------------+-----------------------+-------------------+---------------------+----------------------+--------------------+-------------------+----------------+-------------+--------------------+

<h3 style="color: #1EB382;font-weight: bold;">Derived Feature Group</h3>

Finally users can save the output of their transformations in the feature store as a derived cached feature group.

Storing feature groups as cached feature groups within Hopsworks provides several benefits. First it allows users to leverage HUDI for upserts and time travel capabilities. As new data is ingested, new commits are tracked by Hopsworks allowing users to see what has changed over time.
Similarly to on-demand feature groups, at each commit, statistics are computed and tracked in Hopsworks, allowing users to understand how the data changes.

Cached feature groups can also be made available online (`online_enabled=True`) thus enabling low latency requests to augment inference vectors.

In [116]:
val telcoFg = (fs.createFeatureGroup()
                 .name("telco_customer_features")
                 .version(2)
                 .description("Telecom customer features")
                 .onlineEnabled(true)
                 .timeTravelFormat(TimeTravelFormat.HUDI)
                 .primaryKeys(Seq("customer_id"))
                 .statisticsEnabled(true)
                 .build())

telcoFg: com.logicalclocks.hsfs.FeatureGroup = com.logicalclocks.hsfs.FeatureGroup@6927f5b8


In [117]:
telcoFg.save(telcoFgDf)

An error was encountered:
java.io.IOException: Error: 400{"type":"restApiJsonResponse","errorCode":270121,"errorMsg":"Primary key and partition key are required when using Hudi time travel format"}
  at com.logicalclocks.hsfs.metadata.AuthorizationHandler.handleResponse(AuthorizationHandler.java:45)
  at org.apache.http.impl.client.CloseableHttpClient.execute(CloseableHttpClient.java:223)
  at org.apache.http.impl.client.CloseableHttpClient.execute(CloseableHttpClient.java:191)
  at com.logicalclocks.hsfs.metadata.HopsworksInternalClient.handleRequest(HopsworksInternalClient.java:148)
  at com.logicalclocks.hsfs.metadata.HopsworksClient.handleRequest(HopsworksClient.java:151)
  at com.logicalclocks.hsfs.metadata.FeatureGroupApi.saveInternal(FeatureGroupApi.java:125)
  at com.logicalclocks.hsfs.metadata.FeatureGroupApi.save(FeatureGroupApi.java:105)
  at com.logicalclocks.hsfs.engine.FeatureGroupEngine.saveFeatureGroup(FeatureGroupEngine.java:90)
  at com.logicalclocks.hsfs.FeatureGroup